# This tutorial shows how to train and evaluate surrogate models in bulk on Colab

**Step-1: Load necessary modules**

In [4]:
# run this cell to download data and necessary modules
import os, shutil
repo = 'DeepRFM'
if os.path.isdir(repo):
  shutil.rmtree(repo)
!git clone https://github.com/pinakm9/DeepRFM.git
# add modules folder to Python's search path
import os, sys
from pathlib import Path
script_dir = Path(os.path.dirname(os.path.abspath('')))
sys.path.insert(0, '/content/' + repo + '/modules')

import numpy as np
import utility as ut
import matplotlib.pyplot as plt
import pandas as pd
import rfm, skipRFM, localRFM, localSkip, deepSkip, deepRFM, localDeepRFM, localDeepSkip
import torch
from torch import nn
import l63
import config
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Cloning into 'DeepRFM'...
remote: Enumerating objects: 1049, done.
remote: Counting objects: 100% (1049/1049), done.
remote: Compressing objects: 100% (741/741), done.
remote: Total 1049 (delta 139), reused 1021 (delta 111), pack-reused 0 (from 0)
Receiving objects: 100% (1049/1049), 32.57 MiB | 40.19 MiB/s, done.
Resolving deltas: 100% (139/139), done.
Mounted at /content/gdrive


**Step-2: Set up experiment parameters**
We'll train RFM and SkipRFM models for L63 with $D_r=200$ and $dt=0.01$


In [13]:
dynamical_system = "L63"

# parameters for calculating VPT
prediction_time_config = {"error_threshold": 0.05**2, "dt": 0.01, "Lyapunov_time": 1/0.91}

# parameters for training
train_test_config = {"training_points": int(1e5), "n_repeats": 500}

# parameters for generating training and testing data
data_gen_config = {"dt": prediction_time_config["dt"], "train_seed": 22,\
                   "train_size": 2*train_test_config["training_points"],\
                   "test_seed": 43, "test_num": train_test_config["n_repeats"],\
                   "test_size": 2500, "save_folder": None}

# parameters for searching beta
# the parameters below imply the search in conducted within [1e-11, 1e-6] and each order of magnitude is
# subdivided into 25 equally sized intervals and beta values at the interval endpoints are tested with 5 repitions
beta_config = {"negative_log10_range": [6, 11], "resolution":25, "n_repeats": 5,\
               "training_points": train_test_config["training_points"]}

# parameters for defining the architectures that we want to test
# {"Architecture": [D_r, B, G, I]} --> D_r = width, B = depth, G = local state dimension, I = interaction length
# Since we are not using local models here we set G=I=None
beta_arch_config = {"RFM": [[200, 1, None, None]], "SkipRFM": [[200, 1, None, None]]}

# parameters for defining the architectures that we want to test
# {"Architecture": [D_r, B, G, I]} --> D_r = width, B = depth, G = local state dimension, I = interaction length
# Since we are not using local models here we set G=I=None
arch_config = {"RFM": [[200, 1, None, None]], "SkipRFM": [[200, 1, None, None]]}

# The full configuration looks like this
config = {"prediction_time": prediction_time_config, "train_test": train_test_config, "data_gen": data_gen_config, "beta": beta_config,\
          "arch": arch_config}

**Step-3: Run experiments**

In [14]:
config_id = "config_4" # specifies where to look up beta and store the generated data
device = "cuda" if torch.cuda.is_available() else "cpu"
L0, L1, batch_size = 0.4, 3.5, 100
data_folder = f"{repo}/data"

arch_modules = {'RFM': rfm, 'SkipRFM': skipRFM, 'DeepRFM': deepRFM, 'DeepSkip': deepSkip,\
                'LocalRFM': localRFM, 'LocalSkip': localSkip,\
                'LocalDeepRFM': localDeepRFM, 'LocalDeepSkip': localDeepSkip}

train, test = l63.gen_data(**config['data_gen'])
train, test = torch.tensor(train, device=device), torch.tensor(test, device=device)
# set up architecture and evaluate
for architecture in config['arch']:
    folder = f"{data_folder}/{dynamical_system}/{config_id}/{architecture}"
    beta_data = pd.read_csv(f"{folder}/beta/beta_s.csv") # location to optimal values of beta
    for D_r, B, G, I in config['arch'][architecture]:
        print(f"Working on {dynamical_system}-{architecture}({D_r},{B}) ...")
        beta = float(beta_data[(beta_data['D_r']==D_r) & (beta_data['B']==1)]['beta'].iloc[0])
        drf_args = [D_r, B, L0, L1, train, beta, architecture, f"{folder}/D_r-{D_r}_B-{B}", False]
        batch = rfm.BatchDeepRF(arch_modules[architecture].DeepRF, train, test, *drf_args)
        batch.run(**{**config["train_test"], "batch_size": batch_size, **config['prediction_time']})

Time taken by gen_data is 16.9444 seconds
Working on L63-RFM(200,1) ...
Running experiments for batch 0...
Time taken = 3.13E+01s
Running experiments for batch 1...
Time taken = 2.97E+01s
Running experiments for batch 2...
Time taken = 3.01E+01s
Running experiments for batch 3...
Time taken = 3.02E+01s
Running experiments for batch 4...
Time taken = 3.23E+01s
Time taken by run is 153.6731 seconds
Working on L63-SkipRFM(200,1) ...
Running experiments for batch 0...
Time taken = 3.04E+01s
Running experiments for batch 1...
Time taken = 3.10E+01s
Running experiments for batch 2...
Time taken = 3.18E+01s
Running experiments for batch 3...
Time taken = 3.13E+01s
Running experiments for batch 4...
Time taken = 3.04E+01s
Time taken by run is 154.8507 seconds


**Upload data to your Google Drive (change paths accordingly)**

In [15]:
!zip -r /content/data.zip /content/DeepRFM/data
!cp /content/data.zip "/content/gdrive/My Drive/Pinak/DeepRFM"

  adding: content/DeepRFM/data/ (stored 0%)
  adding: content/DeepRFM/data/L63/ (stored 0%)
  adding: content/DeepRFM/data/L63/config_3_s/ (stored 0%)
  adding: content/DeepRFM/data/L63/config_3_s/SkipRFM/ (stored 0%)
  adding: content/DeepRFM/data/L63/config_3_s/SkipRFM/beta/ (stored 0%)
  adding: content/DeepRFM/data/L63/config_3_s/SkipRFM/beta/beta.csv (deflated 46%)
  adding: content/DeepRFM/data/L63/config_3_s/SkipRFM/beta/beta_s.csv (deflated 45%)
  adding: content/DeepRFM/data/L63/config_3_s/SkipRFM/beta/beta_D_r-16384_B-1.csv (deflated 58%)
  adding: content/DeepRFM/data/L63/config_3_s/SkipRFM/D_r-16384_B-1/ (stored 0%)
  adding: content/DeepRFM/data/L63/config_3_s/SkipRFM/D_r-16384_B-1/batch_data.csv (deflated 62%)
  adding: content/DeepRFM/data/L63/config_3_s/RFM/ (stored 0%)
  adding: content/DeepRFM/data/L63/config_3_s/RFM/beta/ (stored 0%)
  adding: content/DeepRFM/data/L63/config_3_s/RFM/beta/beta.csv (deflated 48%)
  adding: content/DeepRFM/data/L63/config_3_s/RFM/beta/b

**Explore generated data**

In this example the data was saved at DeepRFM/data/config_4/RFM/D_r-200_B-1 and DeepRFM/data/config_4/SkipRFM/D_r-200_B-1, specifically in batch_data.csv files.